- If the last <li> in the "paging" <nav> != "Next" we've reached the end of the search 
- Delete Marketing Blurbs   
    
    
STEPS TO GET PROPERTY DATA:
- OPEN PROPERTY
    - 
SKIP PROP 26

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("https://www.apartments.com/orlando-fl/")

In [ ]:
import time

#Clear out total properties list.
totalProps = []
delay = 10 # delay in seconds
isLastPage = False
currPage = 1

while not isLastPage:
    #Wait until properties are loaded
    try:
        time.sleep(2)
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="placardContainer"]/ul[1]/li/article')))
        print (currPage, "Page is ready to be scraped")
    except TimeoutException:
        print ("Loading took too much time")
    
    #Get properties on the current page
    pageProps = driver.find_elements(By.XPATH, '//*[@id="placardContainer"]/ul[1]/li/article')
    
    for i in range(1, len(pageProps)+1):
        currentElement = f'//*[@id="placardContainer"]/ul[1]/li[{i}]/article'
        currentProperty = driver.find_element(By.XPATH, currentElement)
        currentURL = currentProperty.get_attribute('data-url')
        if currentURL is not None:
            totalProps.append(currentURL)
    
    print(len(totalProps))

    #Get pagination list
    try:
        pagination = driver.find_elements(By.XPATH, '//*[@id="paging"]/ol/li')
        lastPage = pagination[-1]
        
        if lastPage.text == 'Next':
                myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="paging"]/ol/li')))
                driver.find_elements(By.XPATH, '//*[@id="paging"]/ol/li')[-1].click()
                currPage += 1
        else:
            isLastPage = True
    except Exception as e:
        print(e, "Error:")

In [102]:
from selenium.webdriver.common.action_chains import ActionChains

#Getting the data off each property
for currProp in totalProps:
    #Navigating to the Property's URL
    driver.get(currProp)
    time.sleep(2)
    
    #Declaring a new dictionary to store the property's information:
    propInfo = {}
    
    #FIELDS SCRAPED:
        #Property URL
    propURL = currProp
    
        #Property Title
    propName = driver.find_element(By.XPATH, '//*[@id="propertyName"]').text
    
        #Property Address, City, State, and Postal Code
    propAddress = driver.find_element(By.XPATH, '//*[@id="propertyAddressRow"]/div/h2/span[1]').text
    propCity = driver.find_element(By.XPATH, '//*[@id="propertyAddressRow"]/div/h2/span[2]').text
    propState = driver.find_element(By.XPATH, '//*[@id="propertyAddressRow"]/div/h2/span[3]/span[1]').text
    propZIP = driver.find_element(By.XPATH, '//*[@id="propertyAddressRow"]/div/h2/span[3]/span[2]').text
    propFullAddress = propAddress + ', ' + propCity + ', ' + propState + ' ' + propZIP
    
        #Property Coordinates
    #Moving the browser to the Location section so the map gets generated.
    location = driver.find_element(By.XPATH, '//*[@id="mapSection"]/h2')
    actions = ActionChains(driver)
    actions.move_to_element(location).perform()
    
    #Sleep function is called to allow the website to fill out the information
    time.sleep(3)
    coorURL = driver.find_element(By.XPATH, '/html/body/div[1]/main/section[1]/div[1]/div[2]/div[1]/div[2]/section[6]/div[2]/div[1]/div/div[2]/div/div/div[14]/div/a').get_attribute('href')
    print(coorURL)
    

https://maps.google.com/maps?ll=28.46481,-81.45025&z=16&hl=en-US&gl=US&mapclient=apiv3
https://maps.google.com/maps?ll=28.5423,-81.28695&z=16&hl=en-US&gl=US&mapclient=apiv3
https://maps.google.com/maps?ll=28.45456,-81.46157&z=16&hl=en-US&gl=US&mapclient=apiv3
https://maps.google.com/maps?ll=28.45396,-81.28984&z=16&hl=en-US&gl=US&mapclient=apiv3
https://maps.google.com/maps?ll=28.54723,-81.37612&z=16&hl=en-US&gl=US&mapclient=apiv3
https://maps.google.com/maps?ll=28.56489,-81.3717&z=16&hl=en-US&gl=US&mapclient=apiv3
https://maps.google.com/maps?ll=28.41587,-81.22106&z=16&hl=en-US&gl=US&mapclient=apiv3


KeyboardInterrupt: 

LOOP THROUGH THE TOTALPROPS ARRAY 
    ENTER THE URL
        GET THE INFORMATION
        PUT IT INTO THE DICTIONARY
        EXPORT THAT DICTIONARY AS A JSON
